In [ ]:
!tar -xvzf drive/MyDrive/xview2/xview_baseline.tar.gz -C drive/MyDrive/xview2/

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
xview_baseline/10300100E19A4400/36988_no_label.png
xview_baseline/10300100E19A4400/36989_no_label.png
xview_baseline/10300100E19A4400/36990_no_label.png
xview_baseline/10300100E19A4400/36991_no_label.png
xview_baseline/10300100E19A4400/36992_no_label.png
xview_baseline/10300100E19A4400/36993_no_label.png
xview_baseline/10300100E19A4400/36994_no_label.png
xview_baseline/10300100E19A4400/36995_no_label.png
xview_baseline/10300100E19A4400/36996_no_label.png
xview_baseline/10300100E19A4400/36997_no_label.png
xview_baseline/10300100E19A4400/36998_no_label.png
xview_baseline/10300100E19A4400/36999_no_label.png
xview_baseline/10300100E19A4400/37000_no_label.png
xview_baseline/10300100E19A4400/37001_no_label.png
xview_baseline/10300100E19A4400/37002_no_label.png
xview_baseline/10300100E19A4400/37003_no_label.png
xview_baseline/10300100E19A4400/37004_no_label.png
xview_baseline/10300100E19A4400/37005_no_label.png
xview_baseline/

In [ ]:
!pip install tensorflow keras pandas numpy scikit-learn==0.24.2. tqdm pillow opencv-python shapely imblearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 54.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
import math
import random
import time
import datetime
import json
import ast
import argparse
import logging
import shutil
from pathlib import Path
import pandas as pd
import numpy as np
import keras
import shapely
from PIL import Image
import cv2
from tqdm import tqdm

#from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_class_weight

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Add, Input, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
from keras import backend as K
from keras import Sequential
import imblearn
from collections import defaultdict

In [ ]:
from sklearn.metrics import f1_score
from sklearn import metrics

In [ ]:
from sklearn.model_selection import train_test_split

def stratified_train_val_test_split(X, y, test_size=0.2, val_size=0.2, random_state=42):
    # Splitting into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, stratify=y, random_state=random_state)
    
    # Calculating the remaining proportion for validation set
    remaining_size = 1 - test_size
    remaining_val_size = val_size / remaining_size
    
    # Splitting the remaining data into validation and test sets
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=remaining_val_size, stratify=y_train, random_state=random_state)
    
    return X_train, X_val, X_test, y_train, y_val, y_test

["drive/MyDrive/xview2/xview_baseline/" + f for f in os.listdir("drive/MyDrive/xview2/xview_baseline/") if f.endswith('csv')]

['drive/MyDrive/xview2/xview_baseline/10300500D9F8D200_labels.csv',
 'drive/MyDrive/xview2/xview_baseline/10300100E291D100_labels.csv',
 'drive/MyDrive/xview2/xview_baseline/10300100E19A4400_labels.csv',
 'drive/MyDrive/xview2/xview_baseline/train_d100.csv',
 'drive/MyDrive/xview2/xview_baseline/train_d200.csv',
 'drive/MyDrive/xview2/xview_baseline/val_d200.csv',
 'drive/MyDrive/xview2/xview_baseline/val_d100.csv',
 'drive/MyDrive/xview2/xview_baseline/test_d100.csv',
 'drive/MyDrive/xview2/xview_baseline/train_a400.csv',
 'drive/MyDrive/xview2/xview_baseline/test_d200.csv',
 'drive/MyDrive/xview2/xview_baseline/val_a400.csv',
 'drive/MyDrive/xview2/xview_baseline/test_a400.csv']

In [ ]:
 df_d200 = pd.read_csv('drive/MyDrive/xview2/xview_baseline/10300500D9F8D200_labels.csv')
 df_d100 = pd.read_csv('drive/MyDrive/xview2/xview_baseline/10300100E291D100_labels.csv')
 df_a400 = pd.read_csv('drive/MyDrive/xview2/xview_baseline/10300100E19A4400_labels.csv')

 df_d200['img_name'] = [f.replace('/media/julian/Extreme Pro//xview_baseline/10300500D9F8D200/', '') for f in df_d200['img_name']]
 df_d100['img_name'] = [f.replace('/media/julian/Extreme Pro//xview_baseline/10300100E291D100/', '') for f in df_d100['img_name']]
 df_a400['img_name'] = [f.replace('/media/julian/Extreme Pro//xview_baseline/10300100E19A4400/', '') for f in df_a400['img_name']]

damage_intensity_encoding = defaultdict(lambda: 0)                                                                                                                        
damage_intensity_encoding['Destroyed'] = 3
damage_intensity_encoding['Damaged'] = 2
damage_intensity_encoding['Possibly damaged'] = 1
damage_intensity_encoding['No visible damage'] = 0
damage_intensity_encoding['no_label'] = 0

df_d200 = df_d200.replace({"damage_gra" : damage_intensity_encoding })
df_d100 = df_d100.replace({"damage_gra" : damage_intensity_encoding })
df_a400 = df_a400.replace({"damage_gra" : damage_intensity_encoding })

df_d200.rename(columns={'img_name':'uuid', 'damage_gra': 'labels'}, inplace=True)
df_d100.rename(columns={'img_name':'uuid', 'damage_gra': 'labels'}, inplace=True)
df_a400.rename(columns={'img_name':'uuid', 'damage_gra': 'labels'}, inplace=True)

In [ ]:
df_d200.head()

In [ ]:
X_train_d200, X_val_d200, X_test_d200, y_train_d200, y_val_d200, y_test_d200 = stratified_train_val_test_split(df_d200[['uuid']], df_d200['labels'])
X_train_a400, X_val_a400, X_test_a400, y_train_a400, y_val_a400, y_test_a400 = stratified_train_val_test_split(df_a400[['uuid']], df_a400['labels'])
X_train_d100, X_val_d100, X_test_d100, y_train_d100, y_val_d100, y_test_d100 = stratified_train_val_test_split(df_d100[['uuid']], df_d100['labels'])

In [ ]:
print(f'D100 Train size: {len(X_train_d100)}')
print(f'D100 Val size:{len(X_val_d100)}')
print(f'D100 Test size:{len(X_test_d100)}')
print('________________________\n')
print(f'D200 Train size: {len(X_train_d200)}')
print(f'D200 Val size:{len(X_val_d200)}')
print(f'D200 Test size:{len(X_test_d200)}')
print('________________________\n')
print(f'a400 Train size: {len(X_train_a400)}')
print(f'a400 Val size:{len(X_val_a400)}')
print(f'a400 Test size:{len(X_test_a400)}')

D100 Train size: 13434
D100 Val size:4479
D100 Test size:4479
________________________

D200 Train size: 16798
D200 Val size:5600
D200 Test size:5600
________________________

a400 Train size: 11513
a400 Val size:3838
a400 Test size:3838


In [ ]:
train_d100 = X_train_d100.join(y_train_d100)
val_d100 = X_val_d100.join(y_val_d100)
test_d100 = X_test_d100.join(y_test_d100)

train_d200 = X_train_d200.join(y_train_d200)
val_d200 = X_val_d200.join(y_val_d200)
test_d200 = X_test_d200.join(y_test_d200)

train_a400 = X_train_a400.join(y_train_a400)
val_a400 = X_val_a400.join(y_val_a400)
test_a400 = X_test_a400.join(y_test_a400)

In [ ]:
def calculate_desc_stats_rus(df):
    rus = imblearn.under_sampling.RandomUnderSampler(random_state=42, sampling_strategy='majority')
    X_res, y_res = rus.fit_resample(df[['uuid']], df['labels'])
    df = pd.DataFrame({'uuid': X_res['uuid'], 'labels': y_res})
    print(f'Number of samples: {len(df)}')
    print(df.labels.value_counts())
calculate_desc_stats_rus(train_d100)
calculate_desc_stats_rus(train_d200)
calculate_desc_stats_rus(train_a400)

Number of samples: 418
3    188
1    124
0     53
2     53
Name: labels, dtype: int64
Number of samples: 499
3    218
1    161
0     60
2     60
Name: labels, dtype: int64
Number of samples: 473
3    205
1    160
0     54
2     54
Name: labels, dtype: int64


In [ ]:
direc = "drive/MyDrive/xview2/xview_baseline/"

train_d100.to_csv(direc + 'train_d100.csv', index=False)
val_d100.to_csv(direc + 'val_d100.csv', index=False)
test_d100.to_csv(direc + 'test_d100.csv', index=False)

train_d200.to_csv(direc + 'train_d200.csv', index=False)
val_d200.to_csv(direc + 'val_d200.csv', index=False)
test_d200.to_csv(direc + 'test_d200.csv', index=False)

train_a400.to_csv(direc + 'train_a400.csv', index=False)
val_a400.to_csv(direc + 'val_a400.csv', index=False)
test_a400.to_csv(direc + 'test_a400.csv', index=False)

In [ ]:
logging.basicConfig(level=logging.INFO)

# Configurations
NUM_WORKERS = 4
NUM_CLASSES = 4
BATCH_SIZE = 1024
NUM_EPOCHS = 10
LEARNING_RATE = 0.001
RANDOM_SEED = 42
LOG_STEP = 150
LOG_DIR = 'drive/MyDrive/xview2/all_img_balanced/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") 

In [ ]:
def ordinal_loss(y_true, y_pred):
    weights = K.cast(K.abs(K.argmax(y_true, axis=1) - K.argmax(y_pred, axis=1))/(K.int_shape(y_pred)[1] - 1), dtype='float32')
    return (1.0 + weights) * keras.losses.categorical_crossentropy(y_true, y_pred )

In [ ]:
from tensorflow.keras.applications import ResNetRS200

In [ ]:
def generate_xBD_baseline_model():
    # Code from the xView2 baseline repository: 
    # https://github.com/DIUx-xView/xView2_baseline/blob/master/model/model.py
    weights = 'imagenet'
    inputs = Input(shape=(128, 128, 3))

    base_model = ResNetRS200(include_top=False, weights=weights, input_shape=(128, 128, 3))

    for layer in base_model.layers:
        layer.trainable = False

    x = Conv2D(32, (5, 5), strides=(1, 1), padding='same', activation='relu', input_shape=(128, 128, 3))(inputs)
    x = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)(x)

    x = Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)(x)

    x = Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)(x)

    x = Flatten()(x)

    base_resnet = base_model(inputs)
    base_resnet = Flatten()(base_resnet)

    concated_layers = Concatenate()([x, base_resnet])

    concated_layers = Dense(2024, activation='relu')(concated_layers)
    concated_layers = Dense(524, activation='relu')(concated_layers)
    concated_layers = Dense(124, activation='relu')(concated_layers)
    output = Dense(4, activation='relu')(concated_layers)

    model = Model(inputs=inputs, outputs=output)
    return model

In [ ]:
def f1(y_true, y_pred):
  # code from the xView2 baseline repository
  # https://github.com/DIUx-xView/xView2_baseline/blob/master/model/damage_classification.py
  # data augmentation was changed
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    
    
        """
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


###
# Creates data generator for validation set
###
def validation_generator(test_csv, test_dir):
    df = pd.read_csv(test_csv)
    df['uuid'] = df["uuid"].astype(str)
    damage_intensity_encoding = dict()
    damage_intensity_encoding[3] = '3'
    damage_intensity_encoding[2] = '2' 
    damage_intensity_encoding[1] = '1' 
    damage_intensity_encoding[0] = '0'

    df = df.replace({"labels" : damage_intensity_encoding })

    gen = keras.preprocessing.image.ImageDataGenerator(
                             rescale=1/255.)


    return gen.flow_from_dataframe(dataframe=df,
                                   directory=test_dir,
                                   x_col='uuid',
                                   y_col='labels',
                                   batch_size=BATCH_SIZE,
                                   shuffle=False,
                                   seed=RANDOM_SEED,
                                   class_mode="categorical",
                                   target_size=(128, 128))


###
# Applies random transformations to training data
###
def augment_data(df, in_dir):
    damage_intensity_encoding = dict()
    damage_intensity_encoding[3] = '3'
    damage_intensity_encoding[2] = '2' 
    damage_intensity_encoding[1] = '1' 
    damage_intensity_encoding[0] = '0'

    df = df.replace({"labels" : damage_intensity_encoding })
    gen = keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True,
                             vertical_flip=True,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             zoom_range=[0.1,0.2],
                             rescale=1/255.)
    return gen.flow_from_dataframe(dataframe=df,
                                   directory=in_dir,
                                   x_col='uuid',
                                   y_col='labels',
                                   batch_size=BATCH_SIZE,
                                   seed=RANDOM_SEED,
                                   class_mode="categorical",
                                   target_size=(128, 128))


In [ ]:
def train_generator(train_dir):
  train_gen = keras.preprocessing.image.ImageDataGenerator(
      horizontal_flip=True,
      rotation_range=360,
      vertical_flip=True,
      width_shift_range=0.3,
      height_shift_range=0.3,
      rescale=1/255.
  )
  train_gen = train_gen.flow_from_directory(
      directory=train_dir,
      color_mode='rgb',
      batch_size=BATCH_SIZE,
      target_size=(128,128),
      shuffle=True,
      seed=RANDOM_SEED
  )
  return train_gen

def val_generator(val_dir):
  val_gen = keras.preprocessing.image.ImageDataGenerator(
                          rescale=1/255.)

  val_gen = val_gen.flow_from_dataframe(
      directory=val_dir,
      color_mode='rgb',
      batch_size=BATCH_SIZE,
      target_size=(128,128),
      shuffle=True,
      seed=RANDOM_SEED
  )
  return val_gen

In [ ]:
def train_model(train_data, train_csv, val_data, val_csv, model_out, model_in):
  # basically code from the xView2 baseline repository
  # https://github.com/DIUx-xView/xView2_baseline/blob/master/model/damage_classification.py
  # I changed:
  #   - the metrics for experiments for from scratch training (didnt work)
  #   - implemented Random undersampler
  #   - model saving
  #   - parts of the evaluation
    if model_in:
      model = tf.keras.models.load_model(model_in, custom_objects={'ordinal_loss': ordinal_loss})
    else:

      METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      ]

      model = generate_xBD_baseline_model()
      # or try keras.optimizers.legacy.adam.Adam ?
      adam = keras.optimizers.Adam(lr=LEARNING_RATE,
                                    beta_1=0.9,
                                    beta_2=0.999,
                                    decay=0.0,
                                    amsgrad=False)


      model.compile(loss=ordinal_loss, optimizer=adam, metrics=METRICS)



    df = pd.read_csv(train_csv)

    # rus = imblearn.under_sampling.RandomUnderSampler(random_state=42, sampling_strategy='majority')

    # X_res, y_res = rus.fit_resample(df[['uuid']], df['labels'])

    # df = pd.DataFrame({'uuid': X_res['uuid'], 'labels': y_res})

    class_weights = compute_class_weight(class_weight='balanced', 
                                         classes=np.unique(df['labels'].to_list()), 
                                         y=df['labels'].to_list())
    d_class_weights = dict(enumerate(class_weights))

    samples = df['uuid'].count()
    steps = np.ceil(samples/BATCH_SIZE)

    # Augments the training data
    train_gen_flow = augment_data(df, train_data)

    #Set up tensorboard logging
    tensorboard_callbacks = keras.callbacks.TensorBoard(log_dir=LOG_DIR,
                                                        batch_size=BATCH_SIZE)

    
    #Filepath to save model weights
    filepath = model_out + "saved-model-{epoch:02d}-{accuracy:.2f}.hdf5"
    filepath = os.path.dirname(filepath)
    checkpoints = keras.callbacks.ModelCheckpoint(filepath,
                                                    monitor=['loss', 'accuracy', 'tp', 'fp', 'tn', 'fn', 'auc'],
                                                    verbose=1,
                                                    # save_weights_only=False,
                                                    save_best_only=False,
                                                    mode='max')


    #Training begins
    model.fit(train_gen_flow,
                        steps_per_epoch=steps,
                        epochs=NUM_EPOCHS,
                        workers=NUM_WORKERS,
                        use_multiprocessing=False,
                        class_weight=d_class_weights,
                        callbacks=[tensorboard_callbacks, checkpoints],
                        verbose=1)
    model_name = Path(train_csv).stem.replace('train_', '')
    model.save(model_out + f'final_model_{model_name}.hdf5')
    #Evalulate f1 weighted scores on validation set
    validation_gen = validation_generator(val_csv, val_data)

    predictions = model.predict(validation_gen)

    val_trues = validation_gen.classes
    val_pred = np.argmax(predictions, axis=-1)

    f1_weighted = f1_score(val_trues, val_pred, average='weighted')
    print(f1_weighted)

In [ ]:
class Args():
  def __init__(self, folder, name):
    self.train_data = f'drive/MyDrive/xview2/xview_baseline/{folder}/' 
    self.train_csv = f'drive/MyDrive/xview2/xview_baseline/train_{name}.csv'
    self.val_data = f'drive/MyDrive/xview2/xview_baseline/{folder}/'
    self.val_csv = f'drive/MyDrive/xview2/xview_baseline/val_{name}.csv'
    self.model_in = 'drive/MyDrive/xview2/xview2_baseline.hdf5'
    self.model_out = 'drive/MyDrive/xview2/xview_baseline/models/'
    self.test_csv = f'drive/MyDrive/xview2/xview_baseline/test_{name}.csv'
    self.test_data = f'drive/MyDrive/xview2/xview_baseline/{folder}/'
args_d100 = Args('10300100E19A4400', 'd100')
args_d200 = Args('10300500D9F8D200', 'd200')
args_a400 = Args('10300100E19A4400', 'a400')

In [ ]:
train_data = args_a400.train_data
train_csv = args_a400.train_csv
val_data = args_a400.val_data
val_csv = args_a400.val_csv
model_in = args_a400.model_in
model_out = args_a400.model_out
test_data = args_a400.test_data
test_csv = args_a400.test_csv

In [ ]:
#tf.compat.v1.disable_eager_execution()
train_model(train_data, train_csv, val_data, val_csv, model_out, model_in)

Found 11497 validated image filenames belonging to 4 classes.


/usr/local/lib/python3.10/dist-packages/keras/preprocessing/image.py:1137: UserWarning: Found 16 invalid image filename(s) in x_col="uuid". These filename(s) will be ignored.
  warnings.warn(


45/45 [==============================] - ETA: 0s - loss: 20.1757 - accuracy: 0.9636
Epoch 1: saving model to drive/MyDrive/xview2/all_img_balanced/models


45/45 [==============================] - 106s 2s/step - loss: 20.1757 - accuracy: 0.9636
Found 3830 validated image filenames belonging to 4 classes.


/usr/local/lib/python3.10/dist-packages/keras/preprocessing/image.py:1137: UserWarning: Found 8 invalid image filename(s) in x_col="uuid". These filename(s) will be ignored.
  warnings.warn(


15/15 [==============================] - 2843s 203s/step
0.9458967284987587


In [37]:
dataset = {
    'd100': '10300100E19A4400',
    'd200': '10300500D9F8D200',
    'a400': '10300100E19A4400'
 }

for i in dataset:
  args = Args(dataset[i], i)
  train_data = args.train_data
  train_csv = args.train_csv
  val_data = args.val_data
  val_csv = args.val_csv
  model_in = args.model_in
  model_out = args.model_out
  test_data = args.test_data
  test_csv = args.test_csv
  train_model(train_data, train_csv, val_data, val_csv, model_out, model_in)
  model_name = Path(train_csv).stem.replace('train_', '')
  model = tf.keras.models.load_model(
      f'drive/MyDrive/xview2/xview_baseline/models/final_model_{model_name}.hdf5', 
      custom_objects={'ordinal_loss': ordinal_loss})
  validation_gen = validation_generator(test_csv, test_data)
  predictions = model.predict(validation_gen)
  val_trues = validation_gen.classes
  val_pred = np.argmax(predictions, axis=-1)
  print(metrics.confusion_matrix(val_trues, val_pred))
  print(metrics.classification_report(val_trues, val_pred))



Found 8152 validated image filenames belonging to 4 classes.


/usr/local/lib/python3.10/dist-packages/keras/preprocessing/image.py:1137: UserWarning: Found 5282 invalid image filename(s) in x_col="uuid". These filename(s) will be ignored.
  warnings.warn(


Epoch 1/10
14/14 [==============================] - ETA: 0s - loss: 31.2488 - accuracy: 0.9574  
Epoch 1: saving model to drive/MyDrive/xview2/xview_baseline/models


14/14 [==============================] - 1633s 63s/step - loss: 31.2488 - accuracy: 0.9574
Epoch 2/10
14/14 [==============================] - ETA: 0s - loss: 31.4069 - accuracy: 0.9573
Epoch 2: saving model to drive/MyDrive/xview2/xview_baseline/models


14/14 [==============================] - 87s 6s/step - loss: 31.4069 - accuracy: 0.9573
Epoch 3/10
14/14 [==============================] - ETA: 0s - loss: 31.6852 - accuracy: 0.9572
Epoch 3: saving model to drive/MyDrive/xview2/xview_baseline/models


...

/usr/local/lib/python3.10/dist-packages/keras/preprocessing/image.py:1137: UserWarning: Found 8 invalid image filename(s) in x_col="uuid". These filename(s) will be ignored.
  warnings.warn(


4/4 [==============================] - 642s 213s/step
0.9458967284987587
Found 3836 validated image filenames belonging to 4 classes.


/usr/local/lib/python3.10/dist-packages/keras/preprocessing/image.py:1137: UserWarning: Found 2 invalid image filename(s) in x_col="uuid". These filename(s) will be ignored.
  warnings.warn(


4/4 [==============================] - 642s 213s/step
[[3696    0    0    0]
 [  54    0    0    0]
 [  18    0    0    0]
 [  68    0    0    0]]
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      3696
           1       0.00      0.00      0.00        54
           2       0.00      0.00      0.00        18
           3       0.00      0.00      0.00        68

    accuracy                           0.96      3836
   macro avg       0.24      0.25      0.25      3836
weighted avg       0.93      0.96      0.95      3836



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn import metrics
model_name = Path(train_csv).stem.replace('train_', '')
model = tf.keras.models.load_model(f'drive/MyDrive/xview2/xview_baseline/models/final_model_{model_name}.hdf5', custom_objects={'ordinal_loss': ordinal_loss})

# model.load_weights('../-saved-model-10-0.88.hdf5')


In [ ]:
validation_gen = validation_generator(test_csv, test_data)
predictions = model.predict(validation_gen)
val_trues = validation_gen.classes
val_pred = np.argmax(predictions, axis=-1)

Found 3836 validated image filenames belonging to 4 classes.


/usr/local/lib/python3.10/dist-packages/keras/preprocessing/image.py:1137: UserWarning: Found 2 invalid image filename(s) in x_col="uuid". These filename(s) will be ignored.
  warnings.warn(


15/15 [==============================] - 2797s 200s/step
